In [1]:
import os
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import collections
from sklearn.model_selection import train_test_split
from sklearn import metrics
from yellowbrick.classifier import ROCAUC
import xgboost
import shap

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load data 
test = pd.read_csv('data/Case2_Test Set Template.csv')
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 1 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Month Year;Mapped_GCK;Sales_EUR  140 non-null    object
dtypes: object(1)
memory usage: 1.2+ KB


In [3]:
training = pd.read_csv('data/Case2_Sales data.csv', delimiter=';')
training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9802 entries, 0 to 9801
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   DATE        9802 non-null   object
 1   Mapped_GCK  9802 non-null   object
 2   Sales_EUR   9802 non-null   object
dtypes: object(3)
memory usage: 229.9+ KB


In [4]:

Macro = pd.read_csv('data/Case2_Market data.csv', delimiter=';')
Macro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221 entries, 0 to 220
Data columns (total 52 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           221 non-null    object 
 1   China                221 non-null    object 
 2   China.1              221 non-null    object 
 3   France               221 non-null    object 
 4   France.1             221 non-null    object 
 5   Germany              221 non-null    object 
 6   Germany.1            221 non-null    object 
 7   Italy                221 non-null    object 
 8   Italy.1              221 non-null    object 
 9   Japan                221 non-null    object 
 10  Japan.1              221 non-null    object 
 11  Switzerland          220 non-null    object 
 12  Switzerland.1        220 non-null    object 
 13  United Kingdom       221 non-null    object 
 14  United Kingdom.1     203 non-null    object 
 15  United States        221 non-null    obj

In [5]:
Macro.describe(include='all')

,Unnamed: 0,China,China.1,France,France.1,Germany,Germany.1,Italy,Italy.1,Japan,...,production index.10,production index.11,production index.12,production index.13,production index.14,production index.15,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51
count,221,221,221,221,221,221,221,221,221,221,...,220,221,221,221,221,221,0.0,0.0,0.0,0.0
unique,221,132,132,72,58,53,72,84,74,69,...,56,60,98,57,68,47,NaN,NaN,NaN,NaN
top,Index 2010=100 (if not otherwise noted),24,24,95,116,115,129,97,106,106,...,92,119,88,104,97,107,NaN,NaN,NaN,NaN
freq,1,5,5,10,9,15,8,9,9,12,...,16,11,10,15,9,15,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
training.describe(include='all')

,DATE,Mapped_GCK,Sales_EUR
count,9802,9802,9802
unique,1216,14,2609
top,16.04.2021,#1,0
freq,14,1179,7134


In [7]:
# # agreate maket to training
# training = training.merge(Macro, on='YearMonth')

In [ ]:
# duplicates for each feature individually

Macro.duplicated().sum()

0

In [ ]:
training['DATE'].duplicated().sum()

8586

In [11]:
#missing values
training.isnull().sum()


DATE          0
Mapped_GCK    0
Sales_EUR     0
dtype: int64

In [12]:
#missing values
Macro.isnull().sum()

Unnamed: 0               0
China                    0
China.1                  0
France                   0
France.1                 0
Germany                  0
Germany.1                0
Italy                    0
Italy.1                  0
Japan                    0
Japan.1                  0
Switzerland              1
Switzerland.1            1
United Kingdom           0
United Kingdom.1        18
United States            0
United States.1          1
Europe                   0
Europe.1                 0
Unnamed: 19              0
Unnamed: 20              0
Unnamed: 21              0
Unnamed: 22              0
Unnamed: 23              0
Unnamed: 24              0
Unnamed: 25              0
Producer Prices          0
Producer Prices.1       18
Producer Prices.2        0
Producer Prices.3       35
Producer Prices.4        0
Producer Prices.5       23
production index         0
production index.1       0
production index.2       1
production index.3       0
production index.4       0
p

In [16]:
for column in Macro.columns:
    duplicated_values = Macro[column][Macro[column].duplicated(keep=False)]
    if not duplicated_values.empty:
        print(f"\nDuplicates in '{column}':\n{duplicated_values.value_counts()}\n")




Duplicates in 'China':
China
24     5
107    5
69     5
52     4
147    4
130    4
134    4
33     3
244    3
167    3
164    3
174    3
146    3
215    3
114    3
86     3
82     3
74     3
99     3
68     3
46     3
59     3
72     3
56     3
208    2
183    2
128    2
190    2
292    2
241    2
181    2
204    2
227    2
176    2
221    2
179    2
201    2
44     2
182    2
85     2
37     2
38     2
48     2
31     2
51     2
30     2
117    2
122    2
27     2
22     2
95     2
28     2
158    2
173    2
236    2
Name: count, dtype: int64


Duplicates in 'China.1':
China.1
24     5
107    5
69     5
52     4
147    4
130    4
134    4
33     3
244    3
167    3
164    3
174    3
146    3
215    3
114    3
86     3
82     3
74     3
99     3
68     3
46     3
59     3
72     3
56     3
208    2
183    2
128    2
190    2
292    2
241    2
181    2
204    2
227    2
176    2
221    2
179    2
201    2
44     2
182    2
85     2
37     2
38     2
48     2
31     2
51     2
30     2
